In [1]:
%%capture
import numpy as np  
import pandas as pd   

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename));

In [2]:
train_paths = "/kaggle/input/datasets-lemlitbang-sibi-alphabets/SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/training/"

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from PIL import Image
import pandas as pd
import os

def traverse_folders(path):
    for folder_name in sorted(os.listdir(path)):
        print('Folder name: ',folder_name)
        
        img_data = []
        df = pd.DataFrame(columns=['path', 'width', 'height'])
        
        image_files = [file_name for file_name in sorted(os.listdir(os.path.join(path, folder_name))) if file_name.lower().endswith(('.jpg', '.png', '.jpeg'))]
        for image_file in image_files:
            
            img_path=os.path.join(path, folder_name, image_file)
            
            with Image.open(img_path) as img:
                width, height = img.size
            img_data.append({'path': img_path, 'width': width, 'height': height})        

        df = pd.concat([df, pd.DataFrame(img_data)], ignore_index=True)

        print(df['width'].astype(int).describe())
        print(df['height'].astype(int).describe())
        print('------------------')
        
traverse_folders(train_paths)


Folder name:  A
count      42.000000
mean     2074.857143
std       792.123068
min       750.000000
25%      1932.000000
50%      2000.000000
75%      2000.000000
max      4208.000000
Name: width, dtype: float64
count      42.000000
mean     2521.714286
std       878.075342
min      1334.000000
25%      2000.000000
50%      2000.000000
75%      3120.000000
max      4608.000000
Name: height, dtype: float64
------------------
Folder name:  B
count      42.000000
mean     2253.428571
std       811.708140
min       750.000000
25%      1932.000000
50%      2128.000000
75%      2500.000000
max      4208.000000
Name: width, dtype: float64
count      42.000000
mean     2700.285714
std       799.339187
min      1334.000000
25%      2500.000000
50%      2500.000000
75%      3120.000000
max      4608.000000
Name: height, dtype: float64
------------------
Folder name:  C
count      42.000000
mean     2074.857143
std       792.123068
min       750.000000
25%      1932.000000
50%      2000.000000
75